<a href="https://colab.research.google.com/github/LeanderHeuvel/Thesis/blob/main/wave_U_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preparations
 + Free up space
 + Download repository and requirements.

In [ ]:
!rm -r "Wave-U-Net-Pytorch"
!rm -rf "/usr/local/lib/python2.7"
!rm -rf "/swift"
!rm -rf "/tensorflow-2.0.0"
!rm -rf "/tensorflow-1.15.2"
!git clone https://github.com/LeanderHeuvel/Wave-U-Net-Pytorch.git


Cloning into 'Wave-U-Net-Pytorch'...
remote: Enumerating objects: 329, done.
remote: Total 329 (delta 0), reused 0 (delta 0), pack-reused 329
Receiving objects: 100% (329/329), 20.61 MiB | 37.22 MiB/s, done.
Resolving deltas: 100% (175/175), done.


/content


In [ ]:
% cd ..
% cd content/Wave-U-Net-Pytorch
% ls

/
/content/Wave-U-Net-Pytorch
audio_examples/  hdf/     model/        README.md         test.py
checkpoints/     LICENSE  predict.py    requirements.txt  train.py
data/            logs/    __pycache__/  Singularity       utils.py


## Downloading pretrained models


In [ ]:
!wget  -P /content/drive/MyDrive/Neurale-netwerken/Model/models.7z  -O /content/drive/MyDrive/Neurale-netwerken/Model/models.7z https://www.dropbox.com/s/r374hce896g4xlj/models.7z?dl=1
!7z x  /content/drive/MyDrive/Neurale-netwerken/Model/models.7z -o/content/drive/MyDrive/Neurale-netwerken/Model


/content/drive/MyDrive/Neurale-netwerken/Model/models.7z: No such file or directory

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/Neurale-netwerken/Model/                                                         
ERROR: No such file or directory
/content/drive/MyDrive/Neurale-netwerken/Model/models.7z



System ERROR:
Unknown error -2147024894


In [ ]:
Q!N

/bin/bash: N: command not found


## Install dependencies


In [ ]:
!pip3 install -r requirements.txt
#!apt-get install p7zip-full

     |████████████████████████████████| 753.4MB 22kB/s 
     |████████████████████████████████| 4.0MB 55.4MB/s 
     |████████████████████████████████| 512kB 51.9MB/s 
     |████████████████████████████████| 133kB 54.8MB/s 
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101


# (Optional) Training with the model

Mount Google Drive and copy data

In [ ]:
from google.colab import drive

drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
!python3.6 train.py --hdf_dir "/content/drive/MyDrive/Neurale-netwerken/HDF" --load_model "/content/drive/MyDrive/Neurale-netwerken/Checkpoints/checkpoint_6380" --dataset_dir "/content/drive/MyDrive/Neurale-netwerken/musdb18" --musdb_version "compressed" --cuda --checkpoint_dir "/content/drive/MyDrive/Neurale-netwerken/Checkpoints" --log_dir "/content/drive/MyDrive/Neurale-netwerken/Log"

python3.6: can't open file 'train.py': [Errno 2] No such file or directory


# Prediction with the model




In [ ]:
cd /content/Wave-U-Net-Pytorch

/content/Wave-U-Net-Pytorch


In [ ]:
!python3.6 predict.py --load_model /content/drive/MyDrive/Neurale-netwerken/Model/waveunet/model --input "/content/drive/MyDrive/Neurale-netwerken/Thesis/Anouk - converted - kort.wav" --output "/content/drive/MyDrive/Neurale-netwerken/Predictions"

Using valid convolutions with 97961 inputs and 88409 outputs
Loading model from checkpoint /content/drive/MyDrive/Neurale-netwerken/Model/waveunet/model
Step 132065


# Merging Vocals and drums

In [ ]:
!pip install pydub

In [ ]:
import os
import glob
from pydub import AudioSegment

music_dir = '/content/drive/MyDrive/Neurale-netwerken/Predictions'  # Path where the songs are located
extension_list = ('*.wav', '*.mp3')

def match_target_amplitude(sound, target_dBFS):
    change_in_dBFS = target_dBFS - sound.dBFS
    return sound.apply_gain(change_in_dBFS)
other = None
bass = None
other = {}
bass = {}
os.chdir(music_dir)
for extension in extension_list:
    for music in glob.glob(extension):
        wav_filename = os.path.splitext(os.path.basename(music))[0] + '.wav'
        #AudioSegment.from_file(music).export(mp3_filename, format='wav')
        music_name_splitted = wav_filename.split(".")
        basename = music_name_splitted[0]
        if music_name_splitted[1] == "wav_other":
          other[basename] = AudioSegment.from_wav(music_dir+"/"+wav_filename)
        if music_name_splitted[1] == "wav_bass":
          bass[basename] = AudioSegment.from_wav(music_dir+"/"+wav_filename)

for elem in bass:
  remix = None
  bass_frag = bass[elem]
  other_frag = other[elem]
  remix = other_frag.overlay(bass_frag, position = 0)
  file_handle = remix.export(elem+"_other_bass.mp3", format="mp3")
  print("converted ",file_handle)

  # remix = other.overlay(bass, position = 0)
  # #file_handle = remix.export(basename+"_other_drums.mp3", format="mp3")
  # converted.append(basename)
  # other = None
  # bass = None

        
# drums = AudioSegment.from_wav("/content/Wave-U-Net-Pytorch/audio_examples/Beatles_kort.wav_drums.wav")
# vocals = AudioSegment.from_wav("/content/Wave-U-Net-Pytorch/audio_examples/Beatles_kort.wav_vocals.wav")
# bass = AudioSegment.from_wav("/content/Wave-U-Net-Pytorch/audio_examples/Beatles_kort.wav_bass.wav")

# song = drums.overlay(vocals, position=0)
# song2 = song.overlay(bass, position=0)
# song_normalized = match_target_amplitude(song2,-20)
# file_handle = song_normalized.export("Beatles.mp3", format="mp3")

converted  <_io.BufferedRandom name='ExcerptA_other_bass.mp3'>
converted  <_io.BufferedRandom name='ExcerptB_other_bass.mp3'>
converted  <_io.BufferedRandom name='Leonard Cohen, kort_other_bass.mp3'>
converted  <_io.BufferedRandom name='Otis - converted - kort_other_bass.mp3'>
converted  <_io.BufferedRandom name='Papermouth_kort_other_bass.mp3'>
converted  <_io.BufferedRandom name='Anouk - converted - kort_other_bass.mp3'>


In [ ]:
print(bass)

{'ExcerptA': <pydub.audio_segment.AudioSegment object at 0x7f26ba3306a0>, 'ExcerptB': <pydub.audio_segment.AudioSegment object at 0x7f26badb9e80>, 'Leonard Cohen, kort': <pydub.audio_segment.AudioSegment object at 0x7f26badb9eb8>, 'Otis - converted - kort': <pydub.audio_segment.AudioSegment object at 0x7f26d5ddf240>, 'Papermouth_kort': <pydub.audio_segment.AudioSegment object at 0x7f26ba355eb8>, 'Anouk - converted - kort': <pydub.audio_segment.AudioSegment object at 0x7f26ba355f28>}


# Remixing music



In [ ]:
import os
import glob
from pydub import AudioSegment

music_dir = '/content/drive/MyDrive/Neurale-netwerken/Predictions'  # Path where the songs are located
extension_list = ('*.wav', '*.mp3')

def match_target_amplitude(sound, target_dBFS):
    change_in_dBFS = target_dBFS - sound.dBFS
    return sound.apply_gain(change_in_dBFS)

other = {}
bass = {}
vocals = {}
drums = {}
os.chdir(music_dir)
for extension in extension_list:
    for music in glob.glob(extension):
        wav_filename = os.path.splitext(os.path.basename(music))[0] + '.wav'
        #AudioSegment.from_file(music).export(mp3_filename, format='wav')
        music_name_splitted = wav_filename.split(".")
        basename = music_name_splitted[0]
        if music_name_splitted[1] == "wav_other":
          other[basename] = AudioSegment.from_wav(music_dir+"/"+wav_filename)
        if music_name_splitted[1] == "wav_bass":
          bass[basename] = AudioSegment.from_wav(music_dir+"/"+wav_filename)
        if music_name_splitted[1] == "wav_drums":
          drums[basename] = AudioSegment.from_wav(music_dir+"/"+wav_filename)
        if music_name_splitted[1] == "wav_vocals":
          vocals[basename] = AudioSegment.from_wav(music_dir+"/"+wav_filename)
  
for elem in bass:
  remix = None
  #reduce volume of bass and other with - 6db! amazing combo!
  bass_frag = bass[elem] - 6
  other_frag = other[elem] - 6
  drums_frag = drums[elem] + 6
  vocals_frag = vocals[elem] + 6 
  if elem == "Anouk - converted - kort":
    ##Weird Google Drive bug, what's going on? Fixed anyway
    print("helloworld")
    remix = other_frag.overlay(bass_frag, position = 0)
    remix = remix.overlay(drums_frag, position = 0)
    remix = remix.overlay(vocals_frag, position = 0)
    file_handle = remix.export(elem+"_remix2.wav", format="wav")
  else:
    remix = other_frag.overlay(bass_frag, position = 0)
    remix = remix.overlay(drums_frag, position = 0)
    remix = remix.overlay(vocals_frag, position = 0)
    file_handle = remix.export(elem+"_remix2.wav", format="wav")
  print("converted BONIII ",file_handle)

  # remix = other.overlay(bass, position = 0)
  # #file_handle = remix.export(basename+"_other_drums.mp3", format="mp3")
  # converted.append(basename)
  # other = None
  # bass = None

        
# drums = AudioSegment.from_wav("/content/Wave-U-Net-Pytorch/audio_examples/Beatles_kort.wav_drums.wav")
# vocals = AudioSegment.from_wav("/content/Wave-U-Net-Pytorch/audio_examples/Beatles_kort.wav_vocals.wav")
# bass = AudioSegment.from_wav("/content/Wave-U-Net-Pytorch/audio_examples/Beatles_kort.wav_bass.wav")

# song = drums.overlay(vocals, position=0)
# song2 = song.overlay(bass, position=0)
# song_normalized = match_target_amplitude(song2,-20)
# file_handle = song_normalized.export("Beatles.mp3", format="mp3")

converted BONIII  <_io.BufferedRandom name='ExcerptA_remix2.wav'>
converted BONIII  <_io.BufferedRandom name='ExcerptB_remix2.wav'>
converted BONIII  <_io.BufferedRandom name='Leonard Cohen, kort_remix2.wav'>
converted BONIII  <_io.BufferedRandom name='Otis - converted - kort_remix2.wav'>
converted BONIII  <_io.BufferedRandom name='Papermouth_kort_remix2.wav'>
helloworld
converted BONIII  <_io.BufferedRandom name='Anouk - converted - kort_remix2.wav'>


# Evaluation


In [ ]:
!mkdir evaluation

In [ ]:
pip install r128gain

     |████████████████████████████████| 225kB 39.0MB/s 
  Created wheel for r128gain: filename=r128gain-1.0.3-cp36-none-any.whl size=23876 sha256=3ca3b3db7afa0f7ffbdea12a8cd118e82b9d8c8808c2bfbaf25e70566d0090a3
  Stored in directory: /root/.cache/pip/wheels/e6/8d/08/7b3958e6b52839bcc4a0d27910da0f77e5b8fd3425dab3eb30
Successfully built r128gain


In [ ]:
import musdb
import museval
import os
import numpy as np

def estimate_and_evaluate(track):
    # assume mix as estimates
    estimates = {
        'vocals': track.audio,
        'accompaniment': track.audio
    }

    # Evaluate using museval
    scores = museval.eval_mus_track(
        track, estimates, output_dir="evaluation/"
    )

    # print nicely formatted and aggregated scores
    print(scores)


os.environ["MUSDB_PATH"] = "/content/drive/MyDrive/musdb18"
mus = musdb.DB() 
for track in mus:
  print(track)
#for track in mus:
 #   estimate_and_evaluate(track)

In [ ]:
import numpy as np


estimates = {
    'vocals': np.random.random(track.audio.shape),
    'accompaniment': np.random.random(track.audio.shape)
}

print(museval.eval_mus_track(track, estimates, output_dir="evaluation"))

In [ ]:
np.array([])

In [ ]:
!r128gain -d "//content/drive/MyDrive/Neurale-netwerken/Thesis/Anouk - converted - kort.wav"

Unhandled 'WAVE' tag format for file '//content/drive/MyDrive/Neurale-netwerken/Thesis/Anouk - converted - kort.wav'
File '//content/drive/MyDrive/Neurale-netwerken/Thesis/Anouk - converted - kort.wav': loudness = SKIPPED, sample peak = SKIPPED


# Normalize songs


In [ ]:
import os
import glob
from pydub import AudioSegment

music_dir = '/content/drive/MyDrive/Neurale-netwerken/Thesis'  # Path where the videos are located
extension_list = ('*.wav', '*.mp3')

def match_target_amplitude(sound, target_dBFS):
    change_in_dBFS = target_dBFS - sound.dBFS
    return sound.apply_gain(change_in_dBFS)

os.chdir(music_dir)
for extension in extension_list:
    for music in glob.glob(extension):
        mp3_filename = os.path.splitext(os.path.basename(music))[0] + '.wav'
        #AudioSegment.from_file(music).export(mp3_filename, format='wav')

        sound = AudioSegment.from_file(mp3_filename, "wav")
        normalized_sound = match_target_amplitude(sound, -20.0)
        normalized_sound.export(mp3_filename, format="wav")

FileNotFoundError: ignored